In [8]:
import pandas as pd
import numpy as np
from utils import get_names2abbrs_dict

In [9]:
state_po_votes = pd.read_csv('electoral_votes_2020.csv')

In [18]:
d_probs = pd.read_csv('poll_data/all_pres_polls_d_probs.csv')

In [40]:
mask = (d_probs['date'] >= '2020-05-01')
d_probs = d_probs[mask].sort_values(by='date').reset_index(drop=True)

In [41]:
alpha = 0.04
nat_alpha = 0.0
standard_sample_size = 1000

In [42]:
all_states = {k: 0.5 for k in set(get_names2abbrs_dict().values())}  # prior = 0.5 for all states
for row in range(len(d_probs)):
    state_po = d_probs.iloc[row]['state_po']
    sample_size = d_probs.iloc[row]['sample_size'].item()
    d_prob = d_probs.iloc[row]['d_prob'].item()
    sample_weight = (sample_size / standard_sample_size)
    if state_po == 'NAT':
        weight = nat_alpha * sample_weight
        for state_code in all_states.keys():
            all_states[state_code] = (all_states[state_code] * (1 - weight)) + (d_prob * weight)
    else:
        weight = alpha * sample_weight
        all_states[state_po] = (all_states[state_po] * (1 - weight)) + (d_prob * weight)

In [43]:
d_states = list()
r_states = list()
for state_po in all_states:
#     votes = state_po_votes[state_po_votes['state_po'] == state_po]['electoral_votes'].item()
    if all_states[state_po] > 0.5:
        d_states.append(state_po)
    else:
        r_states.append(state_po)
d_votes = state_po_votes[state_po_votes['state_po'].isin(d_states)]['electoral_votes'].sum()
r_votes = state_po_votes[state_po_votes['state_po'].isin(r_states)]['electoral_votes'].sum()

In [45]:
win_str = ''
win_states = None
if d_votes >= 270:
    win_str = f'Biden wins with {d_votes} votes!'
    win_states = d_states
elif r_votes >= 270:
    win_str = f'Trump wins with {r_votes} votes!'
    win_states = r_states
else:
    win_str = f'It\'s a tie! {d_votes} = {r_votes}'
print(win_str)
if win_states:
    print(f'Won with states {sorted(win_states)}')

Biden wins with 350 votes!
Won with states ['AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'MA', 'MD', 'ME', 'MI', 'MN', 'NC', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OR', 'PA', 'RI', 'VA', 'VT', 'WA', 'WI']
